In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import re

from depsurf import RenameType, Version


def get_suffix(x):
    suffix = x.split(".", 1)[-1]
    return re.sub(r"\.\d+", "", suffix)


def count_rename(v: Version):
    all_names = {sym["name"]: sym["bind"] for sym in v.img.symtab.iter_funcs()}
    result = {t: 0 for t in RenameType}

    for func in all_names.keys():
        if "." not in func:
            continue
        suffix = get_suffix(func)
        if suffix == "localalias":
            continue
        result[RenameType.from_suffix(suffix)] += 1

    return {
        **result,
        "Total": len(all_names),
        "GCC": v.img.gcc_version,
    }

In [3]:
from depsurf import VersionGroup
from utils import save_pkl

groups = [VersionGroup.REGULAR, VersionGroup.ARCH]

data = {}
for group in groups:
    group_data = {}
    for version in group.versions:
        group_data[version] = count_rename(version)
    data[group] = group_data

save_pkl(data, "rename")

[        utils.py:160] INFO: NumExpr defaulting to 8 threads.
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.4.0-21-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.8.0-22-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.10.0-19-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.13.0-16-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.15.0-20-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.18.0-10-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/5.0.0-13-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Dow

In [4]:
import pandas as pd

from utils import load_pkl

data = load_pkl("rename")
pd.DataFrame(
    {
        (group, version): data[group][version]
        for group in data
        for version in data[group]
    }
).T

[          pkl.py:18 ] INFO: Loding rename from /Users/szhong/Downloads/bpf-study/output/rename.pkl


isra constprop part  cold   ≥2  Total     GCC
Regular 4.4.0-21-generic-amd64    575       196  522     0   64  38401   5.3.1
        4.8.0-22-generic-amd64    577       233  561     0   63  40274   6.2.0
        4.10.0-19-generic-amd64   588       249  575     0   59  41252   6.3.0
        4.13.0-16-generic-amd64   605       249  724     0   69  43021   7.2.0
        4.15.0-20-generic-amd64   626       257  759     0   65  44662   7.3.0
        4.18.0-10-generic-amd64   706       262  735  1875  161  48438   8.2.0
        5.0.0-13-generic-amd64    717       261  749  1910  157  49281   8.3.0
        5.3.0-18-generic-amd64    782       276  766  4348  330  53711   9.2.1
        5.4.0-26-generic-amd64    780       275  768  3223  244  53486   9.3.0
        5.8.0-25-generic-amd64    602       640  970  3621  369  58346  10.2.0
        5.11.0-16-generic-amd64   623       661  996  3827  393  61863  10.3.0
        5.13.0-19-generic-amd64   516       598  723  3830  312  60245  11.2.0
        5.15.0-25-generic-amd64   510       613  707  8064  548  65484  11.2.0
        5.19.0-21-generic-amd64   547       673  519  8513  506  67567  12.2.0
        6.2.0-20-generic-amd64    562       735  494  5718  420  66993  12.2.0
        6.5.0-9-generic-amd64     813       567  500  6003  420  69967  13.2.0
        6.8.0-31-generic-amd64    851       584  530  6558  435  74048  13.2.0
Arch    5.4.0-26-generic-arm64    967       260  785     0   98  51766   9.3.0
        5.4.0-26-generic-armhf      0       447  804     0    4  50880   9.3.0
        5.4.0-26-generic-ppc64el  589       198  671     0   66  45017   9.3.0
        5.4.0-26-generic-s390x    667       166  532     0   75  35219   9.3.0

In [12]:
from depsurf import RenameType, VersionGroup
from utils import load_pkl, get_legend_handles_labels, save_fig, format_yticks
import matplotlib.pyplot as plt
import numpy as np

COLORS = {
    RenameType.ISRA: "C0",
    RenameType.PART: "C1",
    RenameType.COLD: "C2",
    RenameType.CONSTPROP: "C3",
    RenameType.MULTIPLE: "C4",
}

GROUPS = {
    VersionGroup.REGULAR: "Linux Kernel and GCC Versions",
    VersionGroup.ARCH: "Arch for v5.4\n(GCC 9.3)",
}


data = load_pkl("rename")
fig, axs = plt.subplots(
    1,
    len(data),
    figsize=(5, 1.25),
    sharey=True,
    width_ratios=[len(v) for v in data.values()],
    gridspec_kw={"wspace": 0.1},
)


for group_idx, (ax, (group, group_data)) in enumerate(zip(axs, data.items())):
    for i, (version, version_data) in enumerate(group_data.items()):
        bottom = 0
        for k, v in version_data.items():
            if k in ["Total", "GCC"]:
                continue
            ax.bar(i, v, 0.7, label=k, bottom=bottom, color=COLORS[k])
            bottom += v

        # label total on top
        percentage = f"{bottom / version_data['Total'] * 100:.0f}"
        ax.text(i, bottom, percentage, ha="center", va="bottom", fontsize=8)

    labels = []
    for version, version_data in group_data.items():
        label = f"{group.version_to_str(version)}"
        if group == VersionGroup.REGULAR:
            gcc_version = version_data["GCC"]
            gcc_version = version.img.gcc_version.rsplit(".", 1)[0]
            label += f" / {gcc_version}"
        labels.append(label)
    ax.set_xticks(np.arange(len(labels)), labels)
    ax.set_xlabel(GROUPS[group], fontsize=8)
    ax.set_xticklabels(labels, fontsize=8, rotation=90)
    format_yticks(ax)

    if group == VersionGroup.REGULAR:
        ax.set_ylabel("No. of Functions")

handles, labels = get_legend_handles_labels(fig)
fig.legend(
    handles,
    labels,
    loc="upper center",
    ncol=5,
    bbox_to_anchor=(0.5, 1.1),
    fontsize=8,
    # handlelength=1.5,
    # handletextpad=0.5,
    # columnspacing=1,
)
save_fig(fig, "rename")

[          pkl.py:18 ] INFO: Loding rename from /Users/szhong/Downloads/bpf-study/output/rename.pkl
[          mpl.py:84 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/rename.pdf


4.15 -> 4.18:
- `-freorder-blocks-and-partition` enabed by default in GCC 8

- https://gcc.gnu.org/gcc-8/changes.html

5.0 -> 5.3:
- "Hot/cold partitioning is now more precise and aggressive." since GCC 9

- https://gcc.gnu.org/gcc-9/changes.html

5.13 -> 5.15:

- Non-static functions renamed e.g., `ipv6_local_rxpmtu`

5.19 -> 6.2: 

- `-falign-functions=16` added since Linux 6.2

- https://github.com/torvalds/linux/commit/c27cd083cfb9d392f304657ed00fcde1136704e7

- https://github.com/torvalds/linux/commit/d49a0626216b95cd4bf696f6acf55f39a16ab0bb